In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, RandomizedSearchCV

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

In [3]:
def label_encode(train_data, test_data, columns):
    'Returns a DataFrame with encoded columns'
    encoded_cols = []
    for col in columns:
        factorised = pd.factorize(train_data[col])[1]
        labels = pd.Series(range(len(factorised)), index=factorised)
        encoded_col_train = train_data[col].map(labels) 
        encoded_col_test = test_data[col].map(labels)
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = -1
        encoded_cols.append(pd.DataFrame({'label_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

def freq_encode(train_data, test_data, columns):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    nsamples = train_data.shape[0]
    for col in columns:    
        freqs_cat = train_data.groupby(col)[col].count()/nsamples
        encoded_col_train = train_data[col].map(freqs_cat)
        encoded_col_test = test_data[col].map(freqs_cat)
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = 0
        encoded_cols.append(pd.DataFrame({'freq_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

def mean_encode(train_data, test_data, columns, target_col, reg_method=None,
                alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()
        target_means_cats = train_data.groupby(col)[target_col].mean()
        target_means_cats_adj = (target_means_cats*nrows_cat + 
                                 target_mean_global*alpha)/(nrows_cat+alpha)
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)
        # Getting a train encodings
        if reg_method == 'expanding_mean':
            train_data_shuffled = train_data.sample(frac=1, random_state=1)
            cumsum = train_data_shuffled.groupby(col)[target_col].cumsum() - train_data_shuffled[target_col]
            cumcnt = train_data_shuffled.groupby(col).cumcount()
            encoded_col_train = cumsum/(cumcnt)
            encoded_col_train.fillna(target_mean_global, inplace=True)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))
        elif (reg_method == 'k_fold') and (folds > 1):
            kfold = StratifiedKFold(folds, shuffle=True, random_state=1)
            parts = []
            for tr_in, val_ind in kfold.split(train_data[columns].values, train_data[target_col].values):
                # divide data
                df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
                # getting means on data for estimation (all folds except estimated)
                nrows_cat = df_for_estimation.groupby(col)[target_col].count()
                target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
                target_means_cats_adj = (target_means_cats*nrows_cat + 
                                         target_mean_global*alpha)/(nrows_cat+alpha)
                # Mapping means to estimated fold
                encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
                if add_random:
                    encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, 
                                                                             size=(encoded_col_train_part.shape[0]))
                # Saving estimated encodings for a fold
                parts.append(encoded_col_train_part)
            encoded_col_train = pd.concat(parts, axis=0)
            encoded_col_train.fillna(target_mean_global, inplace=True)
        else:
            encoded_col_train = train_data[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))

        # Saving the column with means
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = target_mean_global
        encoded_cols.append(pd.DataFrame({'mean_'+target_col+'_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

In [4]:
PATH_TO_DATA = Path('../input/flight-delays-fall-2018/')

train_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_train.csv')
test_df = pd.read_csv(PATH_TO_DATA / 'flight_delays_test.csv')
test_df.index = range(100000, 200000)

air = pd.read_csv(PATH_TO_DATA / 'airports.csv')
air = air.set_index('IATA_CODE')

train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [5]:
air_city = air['CITY'].to_dict()
air_state = air['STATE'].to_dict()
air_country = air['COUNTRY'].to_dict()
air_lat = air['LATITUDE'].to_dict()
air_long = air['LONGITUDE'].to_dict()

## Preproc

In [6]:
def upd_time(val):
    hour = val // 100
    minutes = val % 100
    if hour >= 24:
        hour = hour - 24
    return hour * 60 + minutes

def preproc(df, test=False):
    df['Month'] = df['Month'].apply(lambda x: int(x[2:]))
    df['DayofMonth'] = df['DayofMonth'].apply(lambda x: int(x[2:]))
    df['DayOfWeek'] = df['DayOfWeek'].apply(lambda x: int(x[2:]))
    
    df['DepTime'] = df['DepTime'].apply(upd_time)
    
    minutes_in_day = 24 * 60
    df['sin_time'] = np.sin(2 * np.pi * df['DepTime'] / minutes_in_day)
    df['cos_time'] = np.cos(2 * np.pi * df['DepTime'] / minutes_in_day)
    df['hour'] = df['DepTime'] // 60
    df['minutes'] = df['DepTime'] % 60
    
    df['Distance_orig'] = df['Distance'].values
    df['Distance'] = df['Distance'].apply(np.log)

    
    for col in ['Origin', 'Dest']:
        df[col+'CITY'] = df[col].map(air_city).fillna('other')
        df[col+'STATE'] = df[col].map(air_state).fillna('other')
        df[col+'COUNTRY'] = df[col].map(air_country).fillna('other')

        df[col+'LATITUDE'] = df[col].map(air_lat)
        df[col+'LONGITUDE'] = df[col].map(air_long)

        df[col+'x_coord'] = np.cos(df[col+'LATITUDE']) * np.cos(df[col+'LONGITUDE'])
        df[col+'y_coord'] = np.cos(df[col+'LATITUDE']) * np.sin(df[col+'LONGITUDE'])
        df[col+'z_coord'] = np.sin(df[col+'LATITUDE'])
        
    df['x_coord'] = df['Originx_coord'] - df['Destx_coord']
    df['y_coord'] = df['Originy_coord'] - df['Desty_coord']
    df['z_coord'] = df['Originz_coord'] - df['Destz_coord']
    df['LATITUDE'] = df['OriginLATITUDE'] - df['DestLATITUDE']
    df['LONGITUDE'] = df['OriginLONGITUDE'] - df['DestLONGITUDE']
    
    df['flight'] = df['Origin'] + '-->' + df['Dest']
    df['flight_city'] = df['OriginCITY'] + '-->' + df['DestCITY']
    df['flight_state'] = df['OriginSTATE'] + '-->' + df['DestSTATE']
    
    
    df.loc[~df['Origin'].isin(or_dest), 'Origin'] = 'other'
    df.loc[~df['Dest'].isin(or_dest), 'Dest'] = 'other'
    df.loc[~df['UniqueCarrier'].isin(unique_carrier), 'UniqueCarrier'] = 'other'
    df.loc[~df['flight'].isin(flight_set), 'flight'] = 'other'
    df.loc[~df['flight_city'].isin(flight_city), 'flight_city'] = 'other'
    df.loc[~df['flight_state'].isin(flight_state), 'flight_state'] = 'other'
    
    if not test:
        le_carrier.fit(df['UniqueCarrier'])
        le_or_dest.fit(pd.concat([df['Origin'], df['Dest']]))
        le_flight.fit(df['flight'])
        le_flight_city.fit(df['flight_city'])
        le_flight_state.fit(df['flight_state'])
#         del df['dep_delayed_15min']
    
    df['UniqueCarrierLabel'] = le_carrier.transform(df['UniqueCarrier'])
    df['OriginLabel'] = le_or_dest.transform(df['Origin'])
    df['DestLabel'] = le_or_dest.transform(df['Dest'])
    df['flight'] = le_flight.transform(df['flight'])
    df['flight_city'] = le_flight_city.transform(df['flight_city'])
    df['flight_state'] = le_flight_state.transform(df['flight_state'])

    return df

le_carrier = LabelEncoder()
le_or_dest = LabelEncoder()
le_target = LabelEncoder()
le_flight = LabelEncoder()
le_flight_city = LabelEncoder()
le_flight_state = LabelEncoder()

In [7]:
for df in [train_df, test_df]:
    for col in ['Origin', 'Dest']:
        df[col+'CITY'] = df[col].map(air_city).fillna('other')
        df[col+'STATE'] = df[col].map(air_state).fillna('other')
    df['flight'] = df['Origin'] + '-->' + df['Dest']
    df['flight_city'] = df['OriginCITY'] + '-->' + df['DestCITY']
    df['flight_state'] = df['OriginSTATE'] + '-->' + df['DestSTATE']

unique_carrier = list(set(train_df['UniqueCarrier']) & set(test_df['UniqueCarrier']))
or_dest = list(set(train_df['Origin']) & set(test_df['Origin']) & set(train_df['Dest']) & set(test_df['Dest']))
flight_set = list(set(train_df['flight']) & set(test_df['flight']))
flight_city = list(set(train_df['flight_city']) & set(test_df['flight_city']))
flight_state = list(set(train_df['flight_state']) & set(test_df['flight_state']))

In [8]:
y = le_target.fit_transform(train_df['dep_delayed_15min'])
X = preproc(train_df)
X_test = preproc(test_df, test=True)

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,OriginCITY,...,Desty_coord,Destz_coord,x_coord,y_coord,z_coord,LATITUDE,LONGITUDE,UniqueCarrierLabel,OriginLabel,DestLabel
0,8,21,7,1174,AA,ATL,DFW,6.595781,N,Atlanta,...,-0.031268,0.995883,0.646148,0.265882,-0.202054,0.74449,12.61026,0,17,77
1,4,20,3,948,US,PIT,MCO,6.726233,N,Pittsburgh,...,-0.353044,-0.153942,0.808264,-0.579577,0.496120,12.06258,1.08316,15,209,166
2,9,2,5,862,XE,RDU,CLE,6.030685,N,Raleigh,...,0.140751,-0.539788,1.070441,-0.201557,-0.428957,-5.53325,3.06193,17,220,58
3,11,25,6,615,OO,DEN,MEM,6.770789,N,Denver,...,0.799970,-0.466121,-0.075340,-1.265410,1.297894,4.81599,-14.69033,13,76,170
4,10,7,6,1108,WN,MDW,OMA,6.047372,Y,Chicago,...,0.892688,-0.445574,-0.640000,-1.015991,-0.365084,0.48346,8.14175,16,168,194


In [9]:
train_df['dep_delayed_15min'] = y

categ_cols = ['UniqueCarrier', 'Origin', 'Dest', 
              'OriginCITY', 'OriginSTATE', 'OriginCOUNTRY', 
              'DestCITY', 'DestSTATE', 'DestCOUNTRY']
train_df_fe, test_df_fe = freq_encode(train_df, test_df, columns=categ_cols)

train_df_mekf, test_df_mekf = mean_encode(train_df, test_df, columns=categ_cols, target_col='dep_delayed_15min', 
                                                        reg_method='k_fold', alpha=5, folds=5)

train_df_meem, test_df_meem = mean_encode(train_df, test_df, columns=categ_cols, target_col='dep_delayed_15min', 
                                                        reg_method='expanding_mean', alpha=5)

In [10]:
del train_df['dep_delayed_15min']

### CV

In [11]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=38)

In [12]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'Distance',
       'OriginLATITUDE', 'OriginLONGITUDE', 'Originx_coord', 'Originy_coord',
       'Originz_coord', 'DestLONGITUDE', 'Desty_coord', 'x_coord', 'y_coord',
       'z_coord', 'LATITUDE', 'LONGITUDE', 'flight', 'flight_city',
       'flight_state', 'sin_time', 'cos_time', 'minutes',
       'UniqueCarrierLabel']

X_train = X[cols_to_use]

params = {'subsample': 0.55,
         'reg_lambda': 0.35,
         'reg_alpha': 0.7,
         'num_leaves': 147,
         'n_estimators': 743,
         'learning_rate': 0.048496934285281984,
         'class_weight': None}

clf = lgb.LGBMClassifier(random_state=42, silent=True, n_jobs=-1, **params)
cv_score = cross_val_score(clf, X_train, y, cv=skf, scoring='roc_auc')
print(f'{cv_score.mean():.5f}+-{cv_score.std():.5f}')

0.76278+-0.00226


## Voting

In [19]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

In [14]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'Distance',
       'OriginLATITUDE', 'OriginLONGITUDE', 'Originx_coord', 'Originy_coord',
       'Originz_coord', 'DestLONGITUDE', 'Desty_coord', 'x_coord', 'y_coord',
       'z_coord', 'LATITUDE', 'LONGITUDE', 'flight', 'flight_city',
       'flight_state', 'sin_time', 'cos_time', 'minutes',
       'UniqueCarrierLabel']

cols_to_use_cb = ['Month', 'DayofMonth', 'DayOfWeek', 'Distance', 'DepTime', 
               'OriginLATITUDE', 'OriginLONGITUDE', 'DestLATITUDE', 'DestLONGITUDE',
              'UniqueCarrier', 'Origin', 'Dest', 'flight']

X_train = X[cols_to_use]
X_test_ = X_test[cols_to_use]

X_train_cb = X[cols_to_use]
X_test_cb_ = X_test[cols_to_use]

In [15]:
lgb_params = {'subsample': 0.55,
         'reg_lambda': 0.35,
         'reg_alpha': 0.7,
         'num_leaves': 147,
         'n_estimators': 743,
         'learning_rate': 0.048496934285281984,
         'class_weight': None}

cb_params = {'learning_rate': 0.1,
         'leaf_estimation_iterations': 31,
         'l2_leaf_reg': 2310.1297000831582,
         'iterations': 2000,
         'depth': 6,
         'border_count': 254}

categ_feat_idx = np.where(X_train_cb.dtypes == 'object')[0]
fit_params = {'cat_features': categ_feat_idx}

In [16]:
lgb_clf = lgb.LGBMClassifier(random_state=42, silent=True, n_jobs=-1, **lgb_params)
xgb_clf = xgb.XGBClassifier(random_state=42, n_jobs=-1, max_depth=9, n_estimators=250)
cb_clf = CatBoostClassifier(random_state=42, silent=True, **cb_params)

In [39]:
y_pred_lgb = np.zeros(y.shape)
y_pred_xgb = np.zeros(y.shape)
y_pred_cb = np.zeros(y.shape)

for train_idx, test_idx in skf.split(X_train, y):
    print("new split")
    lgb_clf = lgb.LGBMClassifier(random_state=42, silent=True, n_jobs=-1, **lgb_params)
    xgb_clf = xgb.XGBClassifier(random_state=42, n_jobs=-1, max_depth=9, n_estimators=250)
    cb_clf = CatBoostClassifier(random_state=42, silent=True, **cb_params)
    
    lgb_clf.fit(X_train.iloc[train_idx], y[train_idx])
    xgb_clf.fit(X_train.iloc[train_idx], y[train_idx])
    cb_clf.fit(X_train_cb.iloc[train_idx], y[train_idx])
    
    
    y_pred_lgb[test_idx] = lgb_clf.predict_proba(X_train.iloc[test_idx])[:, 1]
    y_pred_xgb[test_idx] = xgb_clf.predict_proba(X_train.iloc[test_idx])[:, 1]
    y_pred_cb[test_idx] = cb_clf.predict_proba(X_train_cb.iloc[test_idx])[:, 1]
    
    
# y_pred_lgb = cross_val_predict(lgb_clf, X_train, y, cv=skf)
# y_pred_xgb = cross_val_predict(xgb_clf, X_train, y, cv=skf)
# y_pred_cb = cross_val_predict(cb_clf, X_train_cb, y, cv=skf)

new split
new split
new split
new split
new split


In [45]:
roc_lgb = roc_auc_score(y, y_pred_lgb)
roc_xgb = roc_auc_score(y, y_pred_xgb)
roc_cb = roc_auc_score(y, y_pred_cb)
print("{:.5f} {:.5f} {:.5f}".format(roc_lgb, roc_xgb, roc_cb))

0.76277 0.75920 0.75214


In [46]:
preds = np.concatenate([y_pred_lgb[:, np.newaxis], y_pred_xgb[:, np.newaxis], y_pred_cb[:, np.newaxis]], axis=1)

In [48]:
y_ens_pred = np.average(preds, axis=1, weights=[0.25, 0.25, 0.50])
roc_auc_score(y, y_ens_pred)

0.7638417637856059

In [53]:
y_ens_pred = np.average(preds, axis=1, weights=[0.33, 0.33, 0.33])
roc_auc_score(y, y_ens_pred)

0.7654257423256483

In [50]:
y_ens_pred = np.average(preds, axis=1, weights=[0.00, 0.00, 1.0])
roc_auc_score(y, y_ens_pred)

0.7521394870833552

Train ens

In [54]:
lgb_clf = lgb.LGBMClassifier(random_state=42, silent=True, n_jobs=-1, **lgb_params)
xgb_clf = xgb.XGBClassifier(random_state=42, n_jobs=-1, max_depth=9, n_estimators=250)
cb_clf = CatBoostClassifier(random_state=42, silent=True, **cb_params)

In [55]:
lgb_clf.fit(X_train,y)
xgb_clf.fit(X_train, y)
cb_clf.fit(X_train_cb, y)

In [56]:
y_pred_lgb = lgb_clf.predict_proba(X_test_)[:, 1]
y_pred_xgb = xgb_clf.predict_proba(X_test_)[:, 1]
y_pred_cb = cb_clf.predict_proba(X_test_cb_)[:, 1]

In [57]:
preds = np.concatenate([y_pred_lgb[:, np.newaxis], y_pred_xgb[:, np.newaxis], y_pred_cb[:, np.newaxis]], axis=1)
y_ens_pred = np.average(preds, axis=1, weights=[0.25, 0.25, 0.50])

In [58]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = y_ens_pred
    sample_sub.to_csv(PATH_TO_DATA / 'ens_pred_clf.csv')

In [ ]:
# Example of hard voting 
from sklearn.ensemble import VotingClassifier

clf = [rf, et, knn, svc, rg]
eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')

for clf, label in zip([rf, et, knn, svc, rg, eclf], ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))# Results

In [ ]:
from itertools import combinations
from sklearn.linear_model import LogisticRegressionlr = LogisticRegression()

names = ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier']

def zip_stacked_classifiers(*args):
    to_zip = []
    for arg in args:
        combined_items = sum([map(list, combinations(arg, i)) for i in range(len(arg) + 1)], [])
        combined_items = filter(lambda x: len(x) > 0, combined_items)
        to_zip.append(combined_items)
    
    return zip(to_zip[0], to_zip[1])

stacked_clf_list = zip_stacked_classifiers(clf_array, names)
best_combination = [0.00, ""]

for clf in stacked_clf_list:
    
    ensemble = SuperLearner(scorer = accuracy_score, 
                            random_state = seed, 
                            folds = 10)
    ensemble.add(clf[0])
    ensemble.add_meta(lr)
    ensemble.fit(X_train, y_train)
    preds = ensemble.predict(X_test)
    accuracy = accuracy_score(preds, y_test)
    
    if accuracy > best_combination[0]:
        best_combination[0] = accuracy
        best_combination[1] = clf[1]
    
    print("Accuracy score: {:.3f} {}").format(accuracy, clf[1])
print("\nBest stacking model is {} with accuracy of: {:.3f}").format(best_combination[1], best_combination[0])

## Train

In [13]:
cols_to_use = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'Distance',
       'OriginLATITUDE', 'OriginLONGITUDE', 'Originx_coord', 'Originy_coord',
       'Originz_coord', 'DestLONGITUDE', 'Desty_coord', 'x_coord', 'y_coord',
       'z_coord', 'LATITUDE', 'LONGITUDE', 'flight', 'flight_city',
       'flight_state', 'sin_time', 'cos_time', 'minutes',
       'UniqueCarrierLabel']

X_train = X[cols_to_use]
X_test_ = X_test[cols_to_use]

In [14]:
clf = lgb.LGBMClassifier(random_state=42, silent=True, n_jobs=-1, **params)
clf.fit(X_train, y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.048496934285281984,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=743, n_jobs=-1, num_leaves=147,
        objective=None, random_state=42, reg_alpha=0.7, reg_lambda=0.35,
        silent=True, subsample=0.55, subsample_for_bin=200000,
        subsample_freq=0)

In [15]:
lgb_test_pred = clf.predict_proba(X_test_)[:, 1]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    sample_sub = pd.read_csv(PATH_TO_DATA / 'sample_submission.csv', 
                             index_col='id')
    sample_sub['dep_delayed_15min'] = lgb_test_pred
    sample_sub.to_csv(PATH_TO_DATA / 'lgbm_pred_clf.csv')

In [17]:
lgb_test_pred

array([0.01402563, 0.01632205, 0.01821385, ..., 0.11643435, 0.05704157,
       0.08139562])